In [ ]:
import pandas as pd
from skyfield.api import EarthSatellite,load
from datetime import datetime, timedelta


pd.set_option('display.max_rows', None)  
pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)  

In [ ]:
df = pd.read_parquet(r'C:\Users\dk412\Desktop\David\Python Projects\RusSat\dataout_HPC\DEV_combat_tle_df.parquet')
df.shape

In [ ]:
df['catalog_number'].nunique()

In [ ]:
df_test = df.iloc[:100]

def build_out_dataframe(dataframe):

    def julian_to_datetime(year, day):
        if year < 50:
            year += 2000
        else:
            year += 1900
            
        base_date = datetime(year, 1, 1)
        days = timedelta(days=day - 1)  
        
        return base_date + days

    df_test['datetime'] = df_test.apply(lambda row: julian_to_datetime(row['epoch_year'], row['epoch_day']), axis=1)

    start_date = pd.to_datetime('1965-01-01')
    end_date = pd.to_datetime('1965-08-01')

    filtered_df = df_test[(df_test['datetime'] >= start_date) & (df_test['datetime'] <= end_date)]
    filtered_df.sort_values(['catalog_number', 'datetime'], inplace=True)
    filtered_df=filtered_df.reset_index(drop=True)

In [ ]:
filtered_df.head()

In [ ]:
def cartesian_pos(dataframe):
    x_lst =[]
    y_lst=[]
    z_lst= []
    coord_lst = []

    for i in range(len(df)):
        L1 = df['line1'].iloc[i]
        L2 = df['line2'].iloc[i]
        sat_nm = str(df['catalog_number'].iloc[i])

        ts = load.timescale()
        satellite = EarthSatellite(L1,L2, sat_nm, ts)
        sat_pos = satellite.at(satellite.epoch)

        x = sat_pos.position.km[0]
        y = sat_pos.position.km[1]
        z = sat_pos.position.km[2]

        x_lst.append(x)
        y_lst.append(y)
        z_lst.append(z)
        coord_lst.append(sat_pos.position.km)
    
    dataframe['x'] = x_lst
    dataframe['y'] = y_lst
    dataframe['z'] = z_lst
    dataframe['cartesian_cords'] = coord_lst


In [ ]:
cartesian_pos(filtered_df)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

def plot_satellite_positions(cord, title="Satellite Positions"):

    x = cord[0]
    y = cord[1]
    z = cord[2]


    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(111, projection='3d')
    
    ax.scatter(x, y, z, c='red', marker='o', s=20, label='Satellite Positions')
    
    #if len(x) > 1:
    #    ax.plot(x, y, z, 'r--', alpha=0.5)
    
    r = 6371  
    phi = np.linspace(0, 2*np.pi, 100)
    theta = np.linspace(0, np.pi, 100)
    phi, theta = np.meshgrid(phi, theta)
    
    x_earth = r * np.cos(phi) * np.sin(theta)
    y_earth = r * np.sin(phi) * np.sin(theta)
    z_earth = r * np.cos(theta)
    
    ax.plot_surface(x_earth, y_earth, z_earth, color='blue', alpha=0.3, label='Earth')
    
    max_range = np.array([x.max()-x.min(), y.max()-y.min(), z.max()-z.min()]).max() / 2.0
    mid_x = (x.max()+x.min()) * 0.5
    mid_y = (y.max()+y.min()) * 0.5
    mid_z = (z.max()+z.min()) * 0.5
    ax.set_xlim(mid_x - max_range, mid_x + max_range)
    ax.set_ylim(mid_y - max_range, mid_y + max_range)
    ax.set_zlim(mid_z - max_range, mid_z + max_range)
    
    ax.set_xlabel('X (km)')
    ax.set_ylabel('Y (km)')
    ax.set_zlabel('Z (km)')
    ax.set_title(title)
    
    ax.grid(True)
    
    return fig, ax

def plot_satellite_trajectory(df, subsample=1, show_earth=True, title="Satellite Trajectory"):

    # Create figure
    fig = plt.figure(figsize=(12, 12))
    ax = fig.add_subplot(111, projection='3d')
    
    # Subsample the data
    x = df['x'].values[::subsample]
    y = df['y'].values[::subsample]
    z = df['z'].values[::subsample]
    
    # Plot trajectory
    ax.plot(x, y, z, 'r-', alpha=0.7, label='Satellite Trajectory')
    
    if show_earth:
        # Create Earth sphere
        r = 6371  # Earth radius in km
        phi = np.linspace(0, 2*np.pi, 100)
        theta = np.linspace(0, np.pi, 100)
        phi, theta = np.meshgrid(phi, theta)
        
        x_earth = r * np.cos(phi) * np.sin(theta)
        y_earth = r * np.sin(phi) * np.sin(theta)
        z_earth = r * np.cos(theta)
        
        # Plot Earth
        ax.plot_surface(x_earth, y_earth, z_earth, color='blue', alpha=0.3, label='Earth')
    
    # Set equal aspect ratio
    max_range = np.array([x.max()-x.min(), y.max()-y.min(), z.max()-z.min()]).max() / 2.0
    mid_x = (x.max()+x.min()) * 0.5
    mid_y = (y.max()+y.min()) * 0.5
    mid_z = (z.max()+z.min()) * 0.5
    ax.set_xlim(mid_x - max_range, mid_x + max_range)
    ax.set_ylim(mid_y - max_range, mid_y + max_range)
    ax.set_zlim(mid_z - max_range, mid_z + max_range)
    
    # Labels and title
    ax.set_xlabel('X (km)')
    ax.set_ylabel('Y (km)')
    ax.set_zlabel('Z (km)')
    ax.set_title(title)
    
    # Add grid
    ax.grid(True)
    
    # Add legend
    ax.legend()
    
    return fig, ax

In [ ]:
# Example usage:
if __name__ == "__main__":
    # For single or multiple discrete points:

    fig, ax = plot_satellite_positions(tle_df['cartesian_cords'].iloc[0], "Satellite Positions")
    plt.show()
    
    fig, ax = plot_satellite_trajectory(tle_df, subsample=10)
    plt.show()

In [ ]:
df= pd.read_parquet(r'D:\Russat\pull\spacetrack_splits\spacetrack_tle_df_E.parquet')

In [ ]:
df.head()

In [ ]:
import pandas as pd

parquets = [r'D:\Russat\pull\spacetrack_splits\spacetrack_tle_df_A.parquet',
            r'D:\Russat\pull\spacetrack_splits\spacetrack_tle_df_B.parquet',
            r'D:\Russat\pull\spacetrack_splits\spacetrack_tle_df_C.parquet',
            r'D:\Russat\pull\spacetrack_splits\spacetrack_tle_df_D.parquet',
            r'D:\Russat\pull\spacetrack_splits\spacetrack_tle_df_E.parquet']

spacetrack_tle_df = pd.DataFrame()

for p in range(len(parquets)):
    df = pd.read_parquet(parquets[p])
    spacetrack_tle_df = pd.concat([spacetrack_tle_df, df], ignore_index = True)
    print(spacetrack_tle_df.shape)


In [ ]:
spacetrack_tle_df.to_parquet(r"C:\Users\dk412\Desktop\David\Python Projects\RusSat\output\spacetrack_tle_df.parquet", engine='pyarrow', compression='gzip', index=True)

In [2]:
#############################################################
#              Prod Code - Filter and xyz add to df COMBAT DATA CORRECTION
#############################################################
import pandas as pd
from skyfield.api import EarthSatellite,load
from datetime import datetime, timedelta



def julian_to_datetime(year, day):
    if year < 50:
        year += 2000
    else:
        year += 1900
        
    base_date = datetime(year, 1, 1)
    days = timedelta(days=day - 1)  
    
    return base_date + days

def build_out_dataframe(df, start_date, end_date):     

    filtered_df = df[(df['datetime'] >= start_date) & (df['datetime'] <= end_date)]
    print("DF filtered to Date Range")

    filtered_df.sort_values(['catalog_number', 'datetime'], inplace=True)
    filtered_df=filtered_df.reset_index(drop=True)
    print("filtered_df:\t", filtered_df.shape)

    x_lst =[]
    y_lst=[]
    z_lst= []
    coord_lst = []

    for i in range(len(filtered_df)):
        L1 = filtered_df['line1'].iloc[i]
        L2 = filtered_df['line2'].iloc[i]
        sat_nm = str(filtered_df['catalog_number'].iloc[i])

        ts = load.timescale()
        satellite = EarthSatellite(L1,L2, sat_nm, ts)
        sat_pos = satellite.at(satellite.epoch)

        x = sat_pos.position.km[0]
        y = sat_pos.position.km[1]
        z = sat_pos.position.km[2]

        x_lst.append(x)
        y_lst.append(y)
        z_lst.append(z)
        coord_lst.append(sat_pos.position.km)
    
    filtered_df['x'] = x_lst
    filtered_df['y'] = y_lst
    filtered_df['z'] = z_lst
    filtered_df['cartesian_cords'] = coord_lst

    return filtered_df

def build_dataset_parquets():
    combat_start_date = pd.to_datetime('2022-02-25')
    combat_end_date = pd.to_datetime('2024-02-24')

    combat_FULL_tle_df = pd.DataFrame()


    for f in range(len(parquets)):

        df = pd.read_parquet(parquets[f])
        df['datetime'] = df.apply(lambda row: julian_to_datetime(row['epoch_year'], row['epoch_day']), axis=1)

        print(f'Working on Combat Data')
        combat_tle_df = build_out_dataframe(df, combat_start_date,combat_end_date)

        print(combat_tle_df.shape)

        combat_FULL_tle_df = pd.concat([combat_FULL_tle_df, combat_tle_df], ignore_index = True)
        print("Full Combat df:\t", combat_FULL_tle_df.shape)

        del df
        del combat_tle_df
        
    return combat_FULL_tle_df 


parquets = [r'D:\Russat\pull\spacetrack_splits\spacetrack_tle_df_A.parquet',
            r'D:\Russat\pull\spacetrack_splits\spacetrack_tle_df_B.parquet',
            r'D:\Russat\pull\spacetrack_splits\spacetrack_tle_df_C.parquet',
            r'D:\Russat\pull\spacetrack_splits\spacetrack_tle_df_D.parquet',
            r'D:\Russat\pull\spacetrack_splits\spacetrack_tle_df_E.parquet']

combat_FULL_tle_df =  build_dataset_parquets()


combat_FULL_tle_df.to_parquet(r"C:\Users\dk412\Desktop\David\Python Projects\RusSat\output\DEV_combat_tle_df.parquet", engine='pyarrow', compression='gzip', index=True)


Working on Combat Data
DF filtered to Date Range


C:\Users\dk412\AppData\Local\Temp\ipykernel_1992\3520530135.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(['catalog_number', 'datetime'], inplace=True)


filtered_df:	 (718506, 24)
(718506, 28)
Full Combat df:	 (718506, 28)
Working on Combat Data
DF filtered to Date Range


C:\Users\dk412\AppData\Local\Temp\ipykernel_1992\3520530135.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(['catalog_number', 'datetime'], inplace=True)


filtered_df:	 (751941, 24)
(751941, 28)
Full Combat df:	 (1470447, 28)
Working on Combat Data
DF filtered to Date Range


C:\Users\dk412\AppData\Local\Temp\ipykernel_1992\3520530135.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(['catalog_number', 'datetime'], inplace=True)


filtered_df:	 (767135, 24)
(767135, 28)
Full Combat df:	 (2237582, 28)
Working on Combat Data
DF filtered to Date Range


C:\Users\dk412\AppData\Local\Temp\ipykernel_1992\3520530135.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.sort_values(['catalog_number', 'datetime'], inplace=True)


filtered_df:	 (1213600, 24)


KeyboardInterrupt: 

In [ ]:
#############################################################
#              Prod Code - Filter and xyz add to df
#############################################################
import pandas as pd
from skyfield.api import EarthSatellite,load
from datetime import datetime, timedelta



def julian_to_datetime(year, day):
    if year < 50:
        year += 2000
    else:
        year += 1900
        
    base_date = datetime(year, 1, 1)
    days = timedelta(days=day - 1)  
    
    return base_date + days

def build_out_dataframe(df, start_date, end_date):     
    #df = pd.read_parquet(input_file)

    df['datetime'] = df.apply(lambda row: julian_to_datetime(row['epoch_year'], row['epoch_day']), axis=1)

    filtered_df = df[(df['datetime'] >= start_date) & (df['datetime'] <= end_date)]
    filtered_df.to_csv(r"C:\Users\dk412\Desktop\David\Python Projects\RusSat\output\datefiltered_combat_tle_df.csv")
    print("DF filtered to Date Range")
    #if filtered_df.empty:
    #    print('No data matching date filter')
    #else:
    filtered_df.sort_values(['catalog_number', 'datetime'], inplace=True)
    filtered_df=filtered_df.reset_index(drop=True)

    x_lst =[]
    y_lst=[]
    z_lst= []
    coord_lst = []

    for i in range(len(filtered_df)):
        L1 = filtered_df['line1'].iloc[i]
        L2 = filtered_df['line2'].iloc[i]
        sat_nm = str(filtered_df['catalog_number'].iloc[i])

        ts = load.timescale()
        satellite = EarthSatellite(L1,L2, sat_nm, ts)
        sat_pos = satellite.at(satellite.epoch)

        x = sat_pos.position.km[0]
        y = sat_pos.position.km[1]
        z = sat_pos.position.km[2]

        x_lst.append(x)
        y_lst.append(y)
        z_lst.append(z)
        coord_lst.append(sat_pos.position.km)
    
    filtered_df['x'] = x_lst
    filtered_df['y'] = y_lst
    filtered_df['z'] = z_lst
    filtered_df['cartesian_cords'] = coord_lst

    return filtered_df

#def build_dataset_parquets():
    #train_start_date = pd.to_datetime('2016-08-24') 
    #train_end_date = pd.to_datetime('2021-08-24') 

    #inf_start_date = pd.to_datetime('2021-08-24')    
    #inf_end_date = pd.to_datetime('2022-02-25')

combat_start_date = pd.to_datetime('2022-02-25')
combat_end_date = pd.to_datetime('2024-02-24')

    #training_FULL_tle_df = pd.DataFrame()
    #inf_FULL_tle_df = pd.DataFrame()
    #combat_FULL_tle_df = pd.DataFrame()

"""
    for f in range(len(parquets)):

        df = pd.read_parquet(parquets[f])

        print(f'Working on Training Data Parquet Number {f}')
        training_tle_df = build_out_dataframe(df, train_start_date,train_end_date)
        print(f'Working on Inference Data Parquet Number {f}')
        inf_tle_df = build_out_dataframe(df, inf_start_date,inf_end_date)
"""
print(f'Working on Combat Data')
combat_tle_df = build_out_dataframe(spacetrack_tle_df, combat_start_date,combat_end_date)

"""
        print(training_tle_df.shape, inf_tle_df.shape, combat_tle_df.shape)

        training_FULL_tle_df = pd.concat([training_FULL_tle_df,training_tle_df], ignore_index = True)
        inf_FULL_tle_df = pd.concat([inf_FULL_tle_df,inf_tle_df], ignore_index = True)
        combat_FULL_tle_df = pd.concat([combat_FULL_tle_df, combat_tle_df], ignore_index = True)

    return training_FULL_tle_df, inf_FULL_tle_df, combat_FULL_tle_df
"""

#training_FULL_tle_df, inf_FULL_tle_df, combat_FULL_tle_df =  build_dataset_parquets()

#training_FULL_tle_df.to_parquet("/homes/dkurtenb/projects/russat/output/DEV_training_tle_df.parquet", engine='pyarrow', compression='gzip', index=True)
#inf_FULL_tle_df.to_parquet("/homes/dkurtenb/projects/russat/output/DEV_inference_tle_df.parquet", engine='pyarrow', compression='gzip', index=True)
combat_tle_df.to_parquet(r"C:\Users\dk412\Desktop\David\Python Projects\RusSat\output\DEV_combat_tle_df.parquet", engine='pyarrow', compression='gzip', index=True)


In [ ]:
print("**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n**\n")